# Prepare core llm chain

In [2]:
import langchain
langchain.debug = False # set to True if  you want to see what the LLM is doing

from langchain.chat_models import AzureChatOpenAI

import dotenv
dotenv.load_dotenv()

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    request_timeout=20,
    max_retries=1,
    client=None,
)

llm.predict('Are you ready?')

'As an AI language model, I am always ready to assist you. How may I help you today?'

In [3]:
from personalizer import *
from response_checker import *
from langchain.prompts.prompt import PromptTemplate

template = """
{prefix}
{goal}: {context}.
{suffix}
"""
prompt = PromptTemplate(
    input_variables=["prefix", "goal", "context", "suffix"],
    template=template,
)
chain = PersonalizerChain.from_llm(
    llm=llm,
    vw_workspace_type=PersonalizerChain.Type.SLATES,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    response_checker = LLMResponseCheckerForSlates(
        llm,
        prompt = ChatPromptTemplate.from_messages(
                [
                    LLMResponseCheckerForSlates.default_system_prompt,
                    HumanMessagePromptTemplate.from_template(
                        '''Given the task:
                           {goal}: {context}
                           rank how good or bad this response is:
                           {llm_response}.'''
                    )]
    ))
)
chain.run({
    'prefix': [f'ALWAYS DO EXACTLY WHAT I ASK YOU!', 'Please do your best to help me.'],
    'goal': 'Write a funny story about',
    'context': 'Friends series',
    'suffix': ['Please try to be as funny as possible.', '']})


2023-07-18 10:35:37,344 - personalizer - INFO - next model checkpoint = 1
2023-07-18 10:35:37,345 - personalizer - INFO - vw command: ['--slates', '--quiet', '--interactions=AC', '--coin', '--squarecb']


'Once upon a time, there were six friends who lived in New York City. They were always getting into hilarious situations, but one day they decided to play a game of "Never Have I Ever."\n\nAs they went around the circle, each friend revealed some pretty wild things they had never done before. Joey had never ridden a rollercoaster, Chandler had never been to a strip club, and Ross had never eaten a sandwich without mayonnaise.\n\nBut when it was Rachel\'s turn, she said, "Never have I ever watched an entire episode of Friends."\n\nThe room fell silent as the other five friends stared at her in shock. "What?!" exclaimed Monica. "How is that even possible?"\n\nRachel shrugged. "I don\'t know, I just never got into it."\n\nThe friends were determined to change that. They spent the next few days binge-watching every episode of Friends with Rachel, making sure she didn\'t miss a single moment of their hilarious antics.\n\nAs they watched, Rachel couldn\'t help but laugh at all the ridiculous

In [ ]:
from personalizer import *
from langchain.prompts.prompt import PromptTemplate

template = """
{prefix}
{goal}: {context}.
{suffix}
"""
prompt = PromptTemplate(
    input_variables=["prefix", "goal", "context", "suffix"],
    template=template,
)
chain = PersonalizerChain.from_llm(
    llm=llm,
    vw_workspace_type=PersonalizerChain.Type.SLATES,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt = prompt,
    response_checker = LLMResponseCheckerForSlates(
        llm,
        prompt = ChatPromptTemplate.from_messages(
                [
                    LLMResponseCheckerForSlates.default_system_prompt,
                    HumanMessagePromptTemplate.from_template(
                        '''Given the task:
                           {goal}: {context}
                           rank how good or bad this response is:
                           {llm_response}.'''
                    )]
    ))
)
chain.run({
    'prefix': Embed([f'ALWAYS DO EXACTLY WHAT I ASK YOU!', 'Please do your best to help me.']),
    'goal': Embed('Write a funny story about'),
    'context': 'Friends series',
    'suffix': ['Please try to be as funny as possible.', '']})